# Predicting Dives from Acceleration Data Using: 

## Tensorflow/Keras

In [22]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
import glob

tf.__version__
keras.__version__

'2.4.0'

### Data Augmentation
1. Load data
2. Look for 1 rows (dives)
3. Augment (create new array from each dive row): 
    * Add noise (2 rows)
    * Shuffle X,Y,Z (6 ways)
    * Bit larger (2 rows)
    * Bit smaller (2 rows)
    * Reverse (1 row)
    * Negate (1 row) 

**Load some data:**

In [ ]:
acc = np.load('../Data/Acc/ch_gps03_S1/6.npz')
acc = acc[acc.files[0]]

**Check for dives:**

In [5]:
(acc[:,-1] == 1).any()

True

**Subset dive rows:**

In [6]:
one_rows = acc[np.where(acc[:,-1] == 1)]
print(one_rows)
print('\nShape:')
print(one_rows.shape)

[[0.25  0.156 0.    ... 1.094 1.141 1.   ]
 [0.156 0.    0.578 ... 1.141 0.953 1.   ]
 [0.    0.578 1.11  ... 0.953 0.953 1.   ]
 ...
 [0.375 0.359 0.297 ... 1.063 1.157 1.   ]
 [0.359 0.297 0.281 ... 1.157 1.11  1.   ]
 [0.297 0.281 0.297 ... 1.11  1.063 1.   ]]

Shape:
(250, 751)


**Augment data**

In [24]:
def augment_row(row, new=200):
    """Create new rows from dive row by adding normally distributed
    noise.
    
    Params:
        row: 1D row np array
        new: integer specifying number of new rows to generate per row
    
    Returns:
        2D np array of augmented data for row.
    """
    row = row[:-1]  # clip off dive value
    
    # Add noise
    noise = np.random.normal(0, .02, (new, len(row)))
    aug = row + noise
    
    return np.hstack((aug, np.ones((new, 1))))


def augment_data(arr, new=200):
    """Create augmented data set. New rows are simply 
    appended on the end of the original dataset.
    
    Params:
        arr: np array containing acceleration and dive data
        new: integer specifying number of new rows to generate per dive row
    
    Returns:
        2D np array of original data stacked on top of augmented data.
    """
    if (acc[:,-1] == 1).any():
        one_rows = arr[np.where(arr[:,-1] == 1)]  # Subset dive rows
    else:
        return arr
    
    new_rows = np.vstack(np.apply_along_axis(augment_row, 0, one_rows, new))
    
    return np.vstack((arr, new_rows))

In [19]:
total_new = augment_data(acc)

print(f'Old data: {acc.shape}')
print(f'New data: {total_new.shape}')
print(f'New rows: {total_new.shape[0]-acc.shape[0]}')

Old data: (489893, 751)
New data: (539893, 751)
New rows: 50000


### Build model

In [38]:
def build_model(in_shape=(750,), l1_units=200, l2_units=200, dropout=0.2):
    """Builds a 2 layer neural network for binary classification with tf.keras.
    """
    # Build model
    model = keras.models.Sequential([
        keras.layers.Dense(units=l1_units, input_shape=in_shape, activation='relu'),
        keras.layers.Dropout(rate=dropout),
        keras.layers.Dense(units=l2_units, activation='relu'),
        keras.layers.Dropout(rate=dropout),
        keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    # Compile model
    model.compile(optimizer = 'rmsprop', 
                  loss = 'binary_crossentropy', 
                  metrics = ['accuracy'])
    return model

model = build_model(in_shape=acc[1,:-1].shape)

### Train model

Here I will use 8/9 birds for training and a single bird for testing.

In [ ]:
t=time.time()

# Grab train and test data
train_files = glob.glob('../Data/Acc/ch_gps03_S1/*.npz')
test_files = glob.glob('../Data/Acc/ch_gps10_S1/*.npz')

for file in train_files:
#//////////////////////
    #file = train_files[21]
    print(file)

    # Load data
    arr = np.load(file)
    arr = arr[arr.files[0]]

    # Augment data
    aug_data = augment_data(arr)

    # exp and resp var
    train_x = aug_data[:,:-1]
    train_y = aug_data[:,-1]

    # Train model
    history = model.fit(train_x, train_y, epochs=20, batch_size=512)
#/////////////////////////

print(time.time()-t)

../Data/Acc/ch_gps03_S1/8.npz
Epoch 1/20
1773/1773 [==============================] - 33s 19ms/step - loss: 0.6882 - accuracy: 0.5231
Epoch 2/20
1773/1773 [==============================] - 30s 17ms/step - loss: 0.0077 - accuracy: 0.5360
Epoch 3/20
1773/1773 [==============================] - 29s 16ms/step - loss: 0.0039 - accuracy: 0.5364
Epoch 4/20
1773/1773 [==============================] - 29s 16ms/step - loss: -0.0042 - accuracy: 0.5364
Epoch 5/20
1773/1773 [==============================] - 28s 16ms/step - loss: -0.0301 - accuracy: 0.5362
Epoch 6/20
1773/1773 [==============================] - 29s 16ms/step - loss: -0.1326 - accuracy: 0.5359
Epoch 7/20
1773/1773 [==============================] - 29s 16ms/step - loss: -0.3884 - accuracy: 0.5354
Epoch 8/20
1773/1773 [==============================] - 30s 17ms/step - loss: -0.9411 - accuracy: 0.5349
Epoch 9/20
1773/1773 [==============================] - 31s 18ms/step - loss: -1.8816 - accuracy: 0.5346
Epoch 10/20
1773/1773 [=====

1365/1365 [==============================] - 24s 18ms/step - loss: -440.3330 - accuracy: 0.6866
Epoch 18/20
1365/1365 [==============================] - 24s 17ms/step - loss: -496.3186 - accuracy: 0.6855
Epoch 19/20
1365/1365 [==============================] - 24s 17ms/step - loss: -539.9907 - accuracy: 0.6841
Epoch 20/20
1365/1365 [==============================] - 23s 17ms/step - loss: -613.5318 - accuracy: 0.6824
../Data/Acc/ch_gps03_S1/17.npz
Epoch 1/20
955/955 [==============================] - 15s 16ms/step - loss: 0.0163 - accuracy: 0.9987
Epoch 2/20
955/955 [==============================] - 15s 16ms/step - loss: 1.8507 - accuracy: 0.9995
Epoch 3/20
955/955 [==============================] - 15s 16ms/step - loss: 0.5294 - accuracy: 0.9996
Epoch 4/20
955/955 [==============================] - 16s 17ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 5/20
955/955 [==============================] - 15s 16ms/step - loss: 0.0259 - accuracy: 0.9997
Epoch 6/20
955/955 [===================

958/958 [==============================] - 14s 15ms/step - loss: 9.0052 - accuracy: 1.0000
Epoch 14/20
958/958 [==============================] - 14s 15ms/step - loss: 6.9807e-04 - accuracy: 1.0000
Epoch 15/20
958/958 [==============================] - 14s 15ms/step - loss: 0.1184 - accuracy: 0.9999
Epoch 16/20
958/958 [==============================] - 15s 16ms/step - loss: 0.3903 - accuracy: 1.0000
Epoch 17/20
958/958 [==============================] - 15s 15ms/step - loss: 0.5528 - accuracy: 1.0000
Epoch 18/20
958/958 [==============================] - 14s 15ms/step - loss: 0.1512 - accuracy: 1.0000
Epoch 19/20
958/958 [==============================] - 14s 15ms/step - loss: 1.3784 - accuracy: 1.0000
Epoch 20/20
958/958 [==============================] - 14s 15ms/step - loss: 1.1031e-04 - accuracy: 1.0000
../Data/Acc/ch_gps03_S1/11.npz
Epoch 1/20
1392/1392 [==============================] - 22s 15ms/step - loss: 26.5330 - accuracy: 0.6358
Epoch 2/20
1392/1392 [======================

2878/2878 [==============================] - 43s 15ms/step - loss: -424.4873 - accuracy: 0.3165
Epoch 10/20
2878/2878 [==============================] - 44s 15ms/step - loss: -544.6112 - accuracy: 0.3167
Epoch 11/20
2878/2878 [==============================] - 44s 15ms/step - loss: -665.5643 - accuracy: 0.3168
Epoch 12/20
2878/2878 [==============================] - 42s 15ms/step - loss: -725.6769 - accuracy: 0.3169
Epoch 13/20
2878/2878 [==============================] - 53s 18ms/step - loss: -925.2432 - accuracy: 0.3164
Epoch 14/20
2878/2878 [==============================] - 43s 15ms/step - loss: -1022.9870 - accuracy: 0.3163
Epoch 15/20
2878/2878 [==============================] - 45s 16ms/step - loss: -1134.1577 - accuracy: 0.3165
Epoch 16/20
2878/2878 [==============================] - 55s 19ms/step - loss: -1353.8228 - accuracy: 0.3161
Epoch 17/20
2878/2878 [==============================] - 51s 18ms/step - loss: -1381.0781 - accuracy: 0.3161
Epoch 18/20
2878/2878 [=============

953/953 [==============================] - 15s 15ms/step - loss: 181.5558 - accuracy: 0.9998
Epoch 6/20
953/953 [==============================] - 16s 17ms/step - loss: 228.5392 - accuracy: 0.9999
Epoch 7/20
953/953 [==============================] - 16s 17ms/step - loss: 238.2781 - accuracy: 0.9999
Epoch 8/20
953/953 [==============================] - 16s 17ms/step - loss: 67.5467 - accuracy: 0.9999
Epoch 9/20
953/953 [==============================] - 16s 17ms/step - loss: 124.4301 - accuracy: 0.9999
Epoch 10/20
953/953 [==============================] - 16s 17ms/step - loss: 125.1679 - accuracy: 0.9999
Epoch 11/20
953/953 [==============================] - 16s 17ms/step - loss: 14.3748 - accuracy: 0.9999
Epoch 12/20
953/953 [==============================] - 16s 17ms/step - loss: 75.2065 - accuracy: 0.9999
Epoch 13/20
953/953 [==============================] - 15s 15ms/step - loss: 63.5194 - accuracy: 0.9999
Epoch 14/20
953/953 [==============================] - 14s 15ms/step - loss

954/954 [==============================] - 15s 16ms/step - loss: 0.0118 - accuracy: 0.9998
Epoch 3/20
954/954 [==============================] - 19s 20ms/step - loss: 0.0129 - accuracy: 0.9999
Epoch 4/20
954/954 [==============================] - 17s 17ms/step - loss: 0.0098 - accuracy: 0.9999
Epoch 5/20
954/954 [==============================] - 15s 16ms/step - loss: 0.0034 - accuracy: 0.9999
Epoch 6/20
954/954 [==============================] - 17s 18ms/step - loss: 0.0013 - accuracy: 0.9999
Epoch 7/20
954/954 [==============================] - 15s 16ms/step - loss: 6.6298e-04 - accuracy: 0.9999
Epoch 8/20
954/954 [==============================] - 15s 16ms/step - loss: 0.0015 - accuracy: 0.9999
Epoch 9/20
954/954 [==============================] - 18s 19ms/step - loss: 0.0031 - accuracy: 0.9999
Epoch 10/20
954/954 [==============================] - 22s 23ms/step - loss: 0.0017 - accuracy: 0.9999
Epoch 11/20
954/954 [==============================] - 15s 15ms/step - loss: 6.2225e-04 

## H20

In [67]:
import numpy as np
import h2o
h2o.init(max_mem_size = "250G")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 day 3 hours 41 mins
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_lukeswaby_petts_6c51o3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,250.0 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [68]:
import time
t = time.time()
data = np.load('../Data/acc_train_data.npy')
df = h2o.H2OFrame(data) 
print(time.time() - t)
# currently: Error: Total input file size of 69.94 GB is much larger than total cluster memory of 1.29 GB, 
# please use either a larger cluster or smaller data.

KeyboardInterrupt: 

# Load each array and save stats to dict with pickle

In [46]:
import glob
import numpy as np
import pickle

dirs = glob.glob('../Data/Acc/*/') 
stats = {}

for dirr in dirs:
    files = glob.glob(f'{dirr}*.npz')
    ones = 0
    zeros = 0
    for file in files:
        dta = np.load(file)
        dta = dta[dta.files[0]]
        no = np.count_nonzero(dta[:,-1] == 0)  # no. of non-dives
        yes = np.count_nonzero(dta[:,-1] == 1)  # no. of dives
        ones += yes
        zeros += no
        del dta
    stats[dirr.split('/')[-2]] = {'Dives': ones, 'Non-dives': zeros, 'Proportion': '%.2f' % ((ones/zeros)*100) + '%'}

filename = '../Data/Acc/birds_dive_stats.pickle'
outfile = open(filename, 'wb')
pickle.dump(stats, outfile)
outfile.close()

### Load file

In [125]:
filename = '../Data/Acc/birds_dive_stats.pickle'
infile = open(filename, 'rb')
bird_stats = pickle.load(infile)
infile.close()

In [65]:
import pandas as pd
pd.DataFrame.from_dict(bird_stats, orient='index')

,Dives,Non-dives,Proportion
ch_gps07_gv37846_20190206_S1,4350,8480502,0.05%
ch_gps11_gv37849_20190206_S1,14750,6894877,0.21%
ch_gps10_S1,5125,9317727,0.06%
ch_gps12_S1,10300,8880802,0.12%
ch_gps16_S1,36275,8654027,0.42%
ch_gps03_S1,33250,12465527,0.27%
ch_gps13_S1,19900,12496252,0.16%
ch_gps08_S1,25525,11317502,0.23%
ch_gps09_S1,8525,9335952,0.09%
